In [ ]:
# to work with datasets on hugging face
!pip install datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
     

In [ ]:
from datasets import Dataset, DatasetDict
from huggingface_hub import HfApi, HfFolder
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
# Data sets with possible questions from user and corresponding SQL query

few_shots=[
    {
        'Question':" Income of last 3 months. ?",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH) """,
        'SQLResult':"1530205868",
        'Answer':"1530205868 is the income of last 3 months."
    }
    ,

    {
        'Question':"Total expenses/spendings of last 8 months ?",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 8 MONTH) """,
        'SQLResult':"11274528407",
        'Answer':"11274528407 is the total expenses of last 8 months."
    },
     {
        'Question':"what is my total expenses of last 3 months ?",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH) """,
        'SQLResult':"11274528407",
        'Answer':"11274528407 is the total expenses of last 8 months."
    },
    {
        'Question':"what is my total expenses of last 4 months for account number 87523?",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Account_No='87523' Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH) """,
        'SQLResult':"11274528407",
        'Answer':"11274528407 is the total expenses of last 8 months."
    },

      {
        'Question':"Total saving of last month as my account number is 409000493201'  ?",
        'SQLQuery':"""SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Savings_Last_Month FROM transactions WHERE Account_No = "409000493201'" AND YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH);
        """,
        'SQLResult':"-193509",
        'Answer':"You lost -193509 last month."
    }
,
    {
        'Question':"How many transactions did I make in last 2 week  ",
        'SQLQuery':"""SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 WEEK) """,
        'SQLResult':"162",
        'Answer':"You made 162 transactions in last 2 week."
    },
     {
        'Question':"Amount spent last week ",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 WEEK) """,
        'SQLResult':"",
        'Answer':"0 is your total spending last week."
    }
    , {
        'Question': "Expenses for each day this month ?",
        'SQLQuery': """SELECT Date, SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) GROUP BY Date """,
        'SQLResult':
        """
Date                        Total_Expenses
2024-06-02                   183648
2024-06-03                   80666
2024-06-04                   135504
2024-06-05                   92031
2024-06-06                   118475
2024-06-07                   16440
2024-06-09                   62112
2024-06-10                   10331
"""
        ,
        'Answer': """The total_expenses is 183648,80666,135504,92031,118475,16440,62112,10331 for the year 2024 month of 6 and date 02,03,04,05,06,07,09 and 10.  """
    },
    {
        'Question':"savings of previous year for each month ",
        'SQLQuery':"""SELECT MONTH(Value_date) AS Month, (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Total_Savings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 YEAR) GROUP BY MONTH(Value_date) ORDER BY MONTH(Value_date) """,
        'SQLResult':"""
        Date                Total_Expenses
        1                       -311244
        2                          9101
        3                         3476
        4                       -15971
        5                       322249
        6                       -172560
        7                       361471
        8                       -446844
        9                       93215
        10                      -73917
        11                      294502
        12                      -404028""",
        'Answer':"Month-1:-311244, Month-2:9101, Month-3:3476"
    },
     {
        'Question':"Amount Saved each year",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year,SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions GROUP BY YEAR(Value_date) """,
        'SQLResult':"""
        Year                Savings
        2021                 497766
        2022                 272458
        2023                 -340550
        2024                 -148114
       """,
        'Answer':"You saved 497766 in 2021 ,272458 in 2022, -340550 in 2023 and -148114 in 2024."
    }
    ,
     {
        'Question':"Savings of last 2 year  ",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Net_Savings FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 YEAR) GROUP BY YEAR(Value_date) """,
        'SQLResult':"""
        Year                Savings
        2022                 2270453
        2023                 -895783
        2024                 -912470
        """,
        'Answer':"You saved 2270453 in 2022, -89578 in 2023 and -912470 in 2024."
    }
    ,
     {
        'Question':"Expenses of each day of last week  ",
        'SQLQuery':"""SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE Value_date >= DATE_SUB(CURDATE(), INTERVAL 1 WEEK) GROUP BY DATE(Value_date) """,
        'SQLResult':"""
        Year                Savings
        2024-06-06           118475
        2024-06-07           16440
        2024-06-09           62112
        2024-06-10           10331

        """,
        'Answer':"You spend 118475,16440,62112,10331."
    },
    {
        'Question':"Amount spent this year ?",
        'SQLQuery':"""SELECT year(Value_date) as Year,sum(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE year(Value_date) = year(curdate()) group by year(Value_date) """,
        'SQLResult':"""
        Year                Total_Expenses
       2024                    6527128485
        """,
        'Answer':"You spent 36616174 this year."
    },
    {
        'Question':"amount/money saved each month previous year ?",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE - INTERVAL 1 YEAR) GROUP BY YEAR(Value_date), MONTH(Value_date) """,
        'SQLResult':"""
        Year            Month              Savings
       2023              1                  -311244
       2023             2                       9101
       2023             3                       3476
       2023             4                       -15971
       2023             5                       322249
       2023             6                       -172560
        2023             7                   361471
       2023             8                       -446844
       2023             9                       93215
       2023             10                       -73917
       2023            11                       294502
       2023             12                       -404028
        """,
        'Answer':"You lost 311244 in month 1 ,saved 9101,3476 in month 2 and month 3  previous year."
    }
     ,
      {
        'Question':"income of the previous year ",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE) - 1 """,
        'SQLResult':"38593214103",
        'Answer':"Your income previous ysear was 38593214103."
    }
    ,
      {
        'Question':"income of the first 4 month of previous year ",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE MONTH(Value_date) IN (1,2,3,4) AND YEAR(Value_date) = YEAR(CURRENT_DATE) - 1 """,
        'SQLResult':"17847505444",
        'Answer':"Your income of first 4 month of previous year was 17847505444."
    },
    {
    'Question': "amount I lost each year?",
    'SQLQuery': """
                SELECT YEAR(Value_date) AS Year,SUM(Withdrawal_amount) - SUM(Deposit_amount) AS Losses
                FROM transactions
                GROUP BY YEAR(Value_date);
                """,
    'SQLResult': """[
        (2022, Decimal('211175202')),
        (2023, Decimal('575324140')),
        (2024, Decimal('490771857')),
        (2021, Decimal('906602921')),
        (2020, Decimal('1076993185'))

    ]"""
    ,
    'Answer': "You lost 497766 in 2021, 272458 in 2022, -340550 in 2023, and -148114 in 2024."
}
  ,
    {
        'Question': "What is the average of last 5 transactions?",
        'SQLQuery': """SELECT AVG(CASE WHEN Withdrawal_amount IS NOT NULL THEN Withdrawal_amount ELSE Deposit_amount END) AS Average_Amount FROM (SELECT Withdrawal_amount, Deposit_amount FROM transactions ORDER BY Date DESC LIMIT 5) AS Last5Transactions"""
    },
    {
        'Question': "What is my total income this year?",
        'SQLQuery': """SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
    },
    {
        'Question': "What is my income of the first 3 months of this year?",
        'SQLQuery': """SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) IN (1, 2, 3)"""
    },
    {
        'Question': "What is my expenses of the first 4 months of this year?",
        'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) IN (1, 2, 3, 4)"""
    },
    {
        'Question': "What is my income of the last 5 months of this year?",
        'SQLQuery': """SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 5 MONTH)"""
    },
    {
        'Question': "How much amount did I earn each year?",
        'SQLQuery': """SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) AS Total_Income FROM transactions GROUP BY YEAR(Value_date)"""
    }
,
    {
        'Question': "How much amount did I save each year?",
        'SQLQuery': """SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions GROUP BY YEAR(Value_date)"""
    },
    {
        'Question': "How much amount did I spend each year?",
        'SQLQuery': """SELECT YEAR(Value_date) AS Year, SUM(Withdrawal_amount) AS Total_Spending FROM transactions GROUP BY YEAR(Value_date)"""
    },
    {
        'Question': "Give me the breakdown of expenses of each day of this week?",
        'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 WEEK) GROUP BY DATE(Value_date)"""
    },
    {
        'Question': "Give me the breakdown of savings of each day of this month?",
        'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Daily_Savings FROM transactions WHERE MONTH(Value_date) = MONTH(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY DATE(Value_date)"""
    },
    {
        'Question': "Give me the breakdown of amount I spent each day of this month?",
        'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending FROM transactions WHERE MONTH(Value_date) = MONTH(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY DATE(Value_date)"""
    },
    {
        'Question': "Give me the breakdown of amount I spent each day of this week?",
        'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 WEEK) GROUP BY DATE(Value_date)"""
    },
    {
        'Question': "On average how much amount I spent each day?",
        'SQLQuery': """SELECT AVG(Withdrawal_amount) AS Average_Daily_Spending FROM transactions"""
    },
    {
        'Question': "On average how much amount I spent each month?",
        'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, AVG(SUM(Withdrawal_amount)) OVER (PARTITION BY YEAR(Value_date), MONTH(Value_date)) AS Average_Monthly_Spending FROM transactions GROUP BY YEAR(Value_date), MONTH(Value_date)"""
    },
    {
        'Question': "On average how much amount I spent each year?",
        'SQLQuery': """SELECT YEAR(Value_date) AS Year, AVG(SUM(Withdrawal_amount)) OVER (PARTITION BY YEAR(Value_date)) AS Average_Yearly_Spending FROM transactions GROUP BY YEAR(Value_date)"""
    },
    {
        'Question': "How many transactions did I make this year?",
        'SQLQuery': """SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
    },
    {
        'Question': "How many transactions did I make each year?",
        'SQLQuery': """SELECT YEAR(Value_date) AS Year, COUNT(*) AS Total_Transactions FROM transactions GROUP BY YEAR(Value_date)"""
    },
    {
        'Question': "How many transactions did I make this week?",
        'SQLQuery': """SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE WEEK(Value_date) = WEEK(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE())"""
    },
    {
        'Question': "How many transactions did I make this month?",
        'SQLQuery': """SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE MONTH(Value_date) = MONTH(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE())"""
    },

    {
        'Question': "What is my total savings of last month?",
        'SQLQuery': """SELECT SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Total_Savings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE - INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE - INTERVAL 1 MONTH)"""
    },
    {
        'Question': "What is my total savings of each year?",
        'SQLQuery': """SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Total_Savings FROM transactions GROUP BY YEAR(Value_date)"""
    },
    {
        'Question': "What is my savings this week?",
        'SQLQuery': """SELECT SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Total_Savings FROM transactions WHERE WEEK(Value_date) = WEEK(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE())"""
    },
    {
        'Question': "What is my expenses amount of last month? ",
        'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE - INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE - INTERVAL 1 MONTH)"""
    },
    {
        'Question': "What is my expenses amount of this month?",
        'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE())"""
    }
,{
    'Question': "What is my current net worth as my account number is 234234",
    'SQLQuery': """SELECT Balance_amount AS Current_Net_Worth FROM transactions where Account_no = "234234" ORDER BY Value_date DESC LIMIT 1"""
}
,{
    'Question': "What is my income of the last 2 years?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Value_date >= DATE_SUB(CURDATE(), INTERVAL 2 YEAR) GROUP BY YEAR(Value_date)"""
},
{
    'Question': "What is my expenses of the last 1 year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Value_date >= DATE_SUB(CURDATE(), INTERVAL 1 YEAR) GROUP BY YEAR(Value_date)"""
},
{
    'Question': "What is my savings of the last 3 years?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Net_Savings FROM transactions WHERE Value_date >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR) GROUP BY YEAR(Value_date)"""
},
    {
    'Question': "Give me the income of the first 7 months of this year",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND MONTH(Value_date) <= 7
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "Tell me the expenses of the first 9 months of this year",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND MONTH(Value_date) <= 9
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "Tell me the income of the last 8 months of this year",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND MONTH(Value_date) >= (MONTH(CURRENT_DATE()) - 7)
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "What is my average spendings each month in this year",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, AVG(Withdrawal_amount) AS Average_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "Give me the average spendings of mine each month in previous year",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, AVG(Withdrawal_amount) AS Average_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "Give me the average spendings of mine each month in year 2022",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, AVG(Withdrawal_amount) AS Average_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = 2022
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "What is my average spendings each week in this month",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, AVG(Withdrawal_amount) AS Average_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND MONTH(Value_date) = MONTH(CURRENT_DATE())
                   GROUP BY WEEK(Value_date)"""
},
{
    'Question': "What is my average spendings each week previous month",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, AVG(Withdrawal_amount) AS Average_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND MONTH(Value_date) = MONTH(CURRENT_DATE()) - 1
                   GROUP BY WEEK(Value_date)"""
},
{
    'Question': "What is my average earning each month in this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, AVG(Deposit_amount) AS Average_Earning
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "Tell me the earnings each month in the previous year",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Earnings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "What is my earnings each month in the year 2018?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Earnings
                   FROM transactions
                   WHERE YEAR(Value_date) = 2018
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "What is my average earning each week in this month?",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, AVG(Deposit_amount) AS Average_Earning
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND MONTH(Value_date) = MONTH(CURRENT_DATE())
                   GROUP BY WEEK(Value_date)"""
},
{
    'Question': "What were my earnings each week in the previous month?",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, SUM(Deposit_amount) AS Total_Earnings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND MONTH(Value_date) = MONTH(CURRENT_DATE()) - 1
                   GROUP BY WEEK(Value_date)"""
},
    {
    'Question': "What was my net worth previous year?",
    'SQLQuery': """SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Net_Worth
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1"""
},
{
    'Question': "What is my net worth this year?",
    'SQLQuery': """SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Net_Worth
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "What was my bank balance in year 2021?",
    'SQLQuery': """SELECT Balance_amount
                   FROM transactions
                   WHERE YEAR(Value_date) = 2021
                   ORDER BY Value_date DESC
                   LIMIT 1"""
},
{
    'Question': "How much money did I spend this year?",
    'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "How much money did I spend till now?",
    'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions"""
},
{
    'Question': "How much money did I spend this week?",
    'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEARWEEK(Value_date, 1) = YEARWEEK(CURRENT_DATE(), 1)"""
},
{
    'Question': "How much money did I spend previous week?",
    'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEARWEEK(Value_date, 1) = YEARWEEK(CURRENT_DATE(), 1) - 1"""
},
{
    'Question': "How much money did I spend this month?",
    'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND MONTH(Value_date) = MONTH(CURRENT_DATE())"""
},
{
    'Question': "How much money did I spend last month?",
    'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND MONTH(Value_date) = MONTH(CURRENT_DATE()) - 1"""
},
    {
    'Question': "What is my highest spending month this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)
                   ORDER BY Total_Spending DESC
                   LIMIT 1"""
},
{
    'Question': "What is my highest earning month this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Earnings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)
                   ORDER BY Total_Earnings DESC
                   LIMIT 1"""
},
{
    'Question': "What is my lowest spending month this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)
                   ORDER BY Total_Spending ASC
                   LIMIT 1"""
},
{
    'Question': "What is my lowest earning month this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Earnings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)
                   ORDER BY Total_Earnings ASC
                   LIMIT 1"""
},
{
    'Question': "What is my average monthly income this year ? ",
    'SQLQuery': """SELECT AVG(Total_Earnings) AS Average_Income
                   FROM (
                       SELECT SUM(Deposit_amount) AS Total_Earnings
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                       GROUP BY MONTH(Value_date)
                   ) AS MonthlyEarnings"""
},
{
    'Question': "What is my average monthly spending this year?",
    'SQLQuery': """SELECT AVG(Total_Spending) AS Average_Spending
                   FROM (
                       SELECT SUM(Withdrawal_amount) AS Total_Spending
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                       GROUP BY MONTH(Value_date)
                   ) AS MonthlySpending"""
},
{
    'Question': "What is my largest transaction this year?",
    'SQLQuery': """SELECT Value_date,MAX(Withdrawal_amount) AS Highest_Transaction_Amount
                    FROM transactions
                    WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                    group by Value_date order by Highest_Transaction_Amount desc limit 1;"""
},
{
    'Question': "What is my smallest transaction this year?",
    'SQLQuery': """SELECT Value_date,Min(Withdrawal_amount) AS Highest_Transaction_Amount
                  FROM transactions
                  WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                  group by Value_date order by Highest_Transaction_Amount desc limit 1;"""
},
{
    'Question': "How many deposits did I make this year?",
    'SQLQuery': """SELECT COUNT(*) AS Total_Deposits
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND Deposit_amount IS NOT NULL"""
},
{
    'Question': "How many withdrawals did I make this year?",
    'SQLQuery': """SELECT COUNT(*) AS Total_Withdrawals
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND Withdrawal_amount IS NOT NULL"""
},
{
    'Question': "What is my total balance at the end of each year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, Balance_amount
                   FROM transactions
                   WHERE Value_date IN (
                       SELECT MAX(Value_date)
                       FROM transactions
                       GROUP BY YEAR(Value_date)
                   )
                   GROUP BY YEAR(Value_date)"""
},
{
    'Question': "What is my highest balance this year?",
    'SQLQuery': """SELECT MAX(Balance_amount) AS Highest_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "What is my lowest balance this year?",
    'SQLQuery': """SELECT MIN(Balance_amount) AS Lowest_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "How many transactions were above $1000 this year?",
    'SQLQuery': """SELECT COUNT(*) AS High_Value_Transactions
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND (Withdrawal_amount > 1000 OR Deposit_amount > 1000)"""
},
{
    'Question': "How many transactions were below $100 this year?",
    'SQLQuery': """SELECT COUNT(*) AS Low_Value_Transactions
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND (Withdrawal_amount < 100 OR Deposit_amount < 100)"""
},
{
    'Question': "What is the average balance in my account this year?",
    'SQLQuery': """SELECT AVG(Balance_amount) AS Average_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "How many days did I have a balance below $500 this year?",
    'SQLQuery': """SELECT COUNT(DISTINCT Value_date) AS Days_Below_500
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND Balance_amount < 500"""
},
{
    'Question': "How many days did I have a balance above $5000 this year?",
    'SQLQuery': """SELECT COUNT(DISTINCT Value_date) AS Days_Above_5000
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND Balance_amount > 5000"""
},
{
    'Question': "How much did I spend on the highest spending day this year?",
    'SQLQuery': """SELECT MAX(Total_Spending) AS Highest_Spending_Day
                   FROM (
                       SELECT SUM(Withdrawal_amount) AS Total_Spending
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                       GROUP BY DATE(Value_date)
                   ) AS DailySpending"""
},
{
    'Question': "How much did I earn on the highest earning day this year?",
    'SQLQuery': """SELECT MAX(Total_Earnings) AS Highest_Earning_Day
                   FROM (
                       SELECT SUM(Deposit_amount) AS Total_Earnings
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                       GROUP BY DATE(Value_date)
                   ) AS DailyEarnings"""
},
    {
    'Question': "What is my total income from last year?",
    'SQLQuery': """SELECT SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1"""
},
{
    'Question': "What is my total spending from last year?",
    'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1"""
},
{
    'Question': "What is the average daily spending in the last month?",
    'SQLQuery': """SELECT AVG(Daily_Spending) AS Average_Daily_Spending
                   FROM (
                       SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending
                       FROM transactions
                       WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)
                       GROUP BY DATE(Value_date)
                   ) AS DailySpending"""
},
{
    'Question': "What is the average daily income in the last month?",
    'SQLQuery': """SELECT AVG(Daily_Income) AS Average_Daily_Income
                   FROM (
                       SELECT DATE(Value_date) AS Date, SUM(Deposit_amount) AS Daily_Income
                       FROM transactions
                       WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)
                       GROUP BY DATE(Value_date)
                   ) AS DailyIncome"""
},
{
    'Question': "What is my largest withdrawal amount this year? as my account number is 409000425051",
    'SQLQuery': """SELECT MAX(Withdrawal_amount) AS Largest_Withdrawal
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   And Account_no='409000425051'"""
},
{
    'Question': "What is my largest deposit amount this year?",
    'SQLQuery': """SELECT MAX(Deposit_amount) AS Largest_Deposit
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "How much did I save in total last year?",
    'SQLQuery': """SELECT SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1"""
},
{
    'Question': "How much did I save in total this year?",
    'SQLQuery': """SELECT SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "How much did I save each month this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Monthly_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "What is the average monthly savings this year?",
    'SQLQuery': """SELECT AVG(Monthly_Savings) AS Average_Monthly_Savings
                   FROM (
                       SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Monthly_Savings
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                       GROUP BY MONTH(Value_date)
                   ) AS MonthlySavings"""
},
{
    'Question': "What is my net income this year?",
    'SQLQuery': """SELECT SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Net_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "What is my net income last year?",
    'SQLQuery': """SELECT SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Net_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1"""
},
{
    'Question': "What is the total number of deposits this year?",
    'SQLQuery': """SELECT COUNT(*) AS Total_Deposits
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND Deposit_amount IS NOT NULL"""
},
{
    'Question': "What is the total number of withdrawals this year?",
    'SQLQuery': """SELECT COUNT(*) AS Total_Withdrawals
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND Withdrawal_amount IS NOT NULL"""
},
{
    'Question': "What is the highest balance in my account this year?",
    'SQLQuery': """SELECT MAX(Balance_amount) AS Highest_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "What is the lowest balance in my account this year?",
    'SQLQuery': """SELECT MIN(Balance_amount) AS Lowest_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "How much did I earn each quarter this year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Deposit_amount) AS Total_Earnings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY QUARTER(Value_date)"""
},
{
    'Question': "How much did I spend each quarter this year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY QUARTER(Value_date)"""
},
{
    'Question': "How much did I save each quarter this year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY QUARTER(Value_date)"""
},
{
    'Question': "How much did I earn each day this week?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Deposit_amount) AS Total_Earnings
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 WEEK)
                   GROUP BY DATE(Value_date)"""
}
    ,
    {
    'Question': "What is my average income each month in the current year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, AVG(Deposit_amount) AS Average_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "What is my total number of transactions each month this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, COUNT(*) AS Total_Transactions
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "What is my average balance each month this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, AVG(Balance_amount) AS Average_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "What is my highest deposit each month this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, MAX(Deposit_amount) AS Highest_Deposit
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "What is my highest withdrawal each month this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, MAX(Withdrawal_amount) AS Highest_Withdrawal
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "How many transactions did I make each quarter this year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, COUNT(*) AS Total_Transactions
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY QUARTER(Value_date)"""
},
{
    'Question': "What is my total income each quarter this year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY QUARTER(Value_date)"""
},
{
    'Question': "What is my total spending each quarter this year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY QUARTER(Value_date)"""
},
{
    'Question': "What is my average balance each quarter this year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, AVG(Balance_amount) AS Average_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY QUARTER(Value_date)"""
},
{
    'Question': "How much did I save each quarter in the last year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY QUARTER(Value_date)"""
},
{
    'Question': "What is my total income this quarter?",
    'SQLQuery': """SELECT SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND QUARTER(Value_date) = QUARTER(CURRENT_DATE())"""
},
{
    'Question': "What is my total spending this quarter?",
    'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND QUARTER(Value_date) = QUARTER(CURRENT_DATE())"""
},
{
    'Question': "What is my total income last quarter?",
    'SQLQuery': """SELECT SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND QUARTER(Value_date) = QUARTER(CURRENT_DATE()) - 1"""
},
{
    'Question': "What is my total spending last quarter?",
    'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND QUARTER(Value_date) = QUARTER(CURRENT_DATE()) - 1"""
},
{
    'Question': "How much did I save last quarter?",
    'SQLQuery': """SELECT SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND QUARTER(Value_date) = QUARTER(CURRENT_DATE()) - 1"""
},
{
    'Question': "What is my highest balance in the last quarter?",
    'SQLQuery': """SELECT MAX(Balance_amount) AS Highest_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND QUARTER(Value_date) = QUARTER(CURRENT_DATE()) - 1"""
},
{
    'Question': "How much did I earn each day in the last 30 days?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Deposit_amount) AS Daily_Earnings
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
                   GROUP BY DATE(Value_date)"""
},
{
    'Question': "How much did I spend each day in the last 30 days?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
                   GROUP BY DATE(Value_date)"""
},
{
    'Question': "What is my average income each day in the last 30 days?",
    'SQLQuery': """SELECT AVG(Daily_Earnings) AS Average_Daily_Income
                   FROM (
                       SELECT DATE(Value_date) AS Date, SUM(Deposit_amount) AS Daily_Earnings
                       FROM transactions
                       WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
                       GROUP BY DATE(Value_date)
                   ) AS DailyIncome"""
},
{
    'Question': "What is my average spending each day in the last 30 days?",
    'SQLQuery': """SELECT AVG(Daily_Spending) AS Average_Daily_Spending
                   FROM (
                       SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending
                       FROM transactions
                       WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
                       GROUP BY DATE(Value_date)
                   ) AS DailySpending"""
}
    ,
    {
    'Question': "What is my highest balance ever recorded?",
    'SQLQuery': """SELECT MAX(Balance_amount) AS Highest_Balance
                   FROM transactions"""
},
{
    'Question': "What is my lowest balance ever recorded?",
    'SQLQuery': """SELECT MIN(Balance_amount) AS Lowest_Balance
                   FROM transactions"""
},
{
    'Question': "What is the total number of transactions this year?",
    'SQLQuery': """SELECT COUNT(*) AS Total_Transactions
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "How much did I earn each week this month?",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, SUM(Deposit_amount) AS Weekly_Earnings
                   FROM transactions
                   WHERE MONTH(Value_date) = MONTH(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY WEEK(Value_date)"""
},
{
    'Question': "How much did I spend each week this month?",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, SUM(Withdrawal_amount) AS Weekly_Spending
                   FROM transactions
                   WHERE MONTH(Value_date) = MONTH(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY WEEK(Value_date)"""
},
{
    'Question': "What is the total number of deposits made this year?",
    'SQLQuery': """SELECT COUNT(*) AS Total_Deposits
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND Deposit_amount > 0"""
},
{
    'Question': "What is the total number of withdrawals made this year?",
    'SQLQuery': """SELECT COUNT(*) AS Total_Withdrawals
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND Withdrawal_amount > 0"""
},
{
    'Question': "What is my total balance at the end of each quarter this year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, QUARTER(Value_date) AS Quarter, Balance_amount
                   FROM transactions
                   WHERE Value_date IN (
                       SELECT MAX(Value_date)
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                       GROUP BY YEAR(Value_date), QUARTER(Value_date)
                   )
                   GROUP BY YEAR(Value_date), QUARTER(Value_date)"""
},
{
    'Question': "How much did I save each month this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Monthly_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "How much did I earn each quarter in the previous year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Deposit_amount) AS Quarterly_Earnings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY QUARTER(Value_date)"""
},
{
    'Question': "How much did I spend each quarter in the previous year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Withdrawal_amount) AS Quarterly_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY QUARTER(Value_date)"""
},
{
    'Question': "What is the average amount per transaction this year?",
    'SQLQuery': """SELECT AVG(Withdrawal_amount + Deposit_amount) AS Average_Transaction_Amount
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "What is the highest deposit made this year?",
    'SQLQuery': """SELECT MAX(Deposit_amount) AS Highest_Deposit
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "What is the highest withdrawal made this year?",
    'SQLQuery': """SELECT MAX(Withdrawal_amount) AS Highest_Withdrawal
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "How much did I save last year?",
    'SQLQuery': """SELECT SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1"""
},
{
    'Question': "How much did I earn each day this week?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Deposit_amount) AS Daily_Earnings
                   FROM transactions
                   WHERE YEARWEEK(Value_date, 1) = YEARWEEK(CURRENT_DATE(), 1)
                   GROUP BY DATE(Value_date)"""
},
{
    'Question': "How much did I spend each day this week?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending
                   FROM transactions
                   WHERE YEARWEEK(Value_date, 1) = YEARWEEK(CURRENT_DATE(), 1)
                   GROUP BY DATE(Value_date)"""
},
{
    'Question': "What is my average daily balance this month?",
    'SQLQuery': """SELECT AVG(Balance_amount) AS Average_Daily_Balance
                   FROM transactions
                   WHERE MONTH(Value_date) = MONTH(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "How many transactions did I make each day this month?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, COUNT(*) AS Total_Transactions
                   FROM transactions
                   WHERE MONTH(Value_date) = MONTH(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY DATE(Value_date)"""
},
{
    'Question': "What is the average amount per transaction this month?",
    'SQLQuery': """SELECT AVG(Withdrawal_amount + Deposit_amount) AS Average_Transaction_Amount
                   FROM transactions
                   WHERE MONTH(Value_date) = MONTH(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE())"""
}
,


{
    'Question': "What is the total number of transactions made last year?",
    'SQLQuery': """SELECT COUNT(*) AS Total_Transactions
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1"""
},
{
    'Question': "What is the total number of transactions made each month last year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, COUNT(*) AS Total_Transactions
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "How much did I earn each week last month?",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, SUM(Deposit_amount) AS Weekly_Earnings
                   FROM transactions
                   WHERE MONTH(Value_date) = MONTH(DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)) AND YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY WEEK(Value_date)"""
},
{
    'Question': "How much did I spend each week last month?",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, SUM(Withdrawal_amount) AS Weekly_Spending
                   FROM transactions
                   WHERE MONTH(Value_date) = MONTH(DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)) AND YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY WEEK(Value_date)"""
},
{
    'Question': "What is my highest spending in a single transaction this year?",
    'SQLQuery': """SELECT MAX(Withdrawal_amount) AS Highest_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "What is my highest earning in a single transaction this year?",
    'SQLQuery': """SELECT MAX(Deposit_amount) AS Highest_Earning
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "What is my lowest spending in a single transaction this year?",
    'SQLQuery': """SELECT MIN(Withdrawal_amount) AS Lowest_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "What is my lowest earning in a single transaction this year?",
    'SQLQuery': """SELECT MIN(Deposit_amount) AS Lowest_Earning
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"""
},
{
    'Question': "What is my average balance each month this year?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, AVG(Balance_amount) AS Average_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date)"""
},
{
    'Question': "What is my total earnings for each day this month?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Deposit_amount) AS Total_Earnings
                   FROM transactions
                   WHERE MONTH(Value_date) = MONTH(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY DATE(Value_date)"""
},
{
    'Question': "What is my total spendings for each day last month?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Total_Spendings
                   FROM transactions
                   WHERE MONTH(Value_date) = MONTH(DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)) AND YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY DATE(Value_date)"""
},
{
    'Question': "What is my highest balance at the end of each month last year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, MAX(Balance_amount) AS Highest_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What is my lowest balance at the end of each month last year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, MIN(Balance_amount) AS Lowest_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What is the total number of transactions made each week this month?",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, COUNT(*) AS Total_Transactions
                   FROM transactions
                   WHERE MONTH(Value_date) = MONTH(CURRENT_DATE()) AND YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY WEEK(Value_date)"""
},
{
    'Question': "What is my total earnings for each week this year?",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, SUM(Deposit_amount) AS Weekly_Earnings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY WEEK(Value_date)"""
},
{
    'Question': "What is my total spendings for each week this year?",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, SUM(Withdrawal_amount) AS Weekly_Spendings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY WEEK(Value_date)"""
},
{
    'Question': "How many transactions did I make each day this week?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, COUNT(*) AS Total_Transactions
                   FROM transactions
                   WHERE YEARWEEK(Value_date, 1) = YEARWEEK(CURRENT_DATE(), 1)
                   GROUP BY DATE(Value_date)"""
},
{
    'Question': "How many transactions did I make each day last week?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, COUNT(*) AS Total_Transactions
                   FROM transactions
                   WHERE YEARWEEK(Value_date, 1) = YEARWEEK(DATE_SUB(CURRENT_DATE(), INTERVAL 1 WEEK), 1)
                   GROUP BY DATE(Value_date)"""
},
{
    'Question': "What is my average balance each week this year?",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, AVG(Balance_amount) AS Average_Balance
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY WEEK(Value_date)"""
},
{
    'Question': "What is my total balance at the end of each quarter last year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, QUARTER(Value_date) AS Quarter, Balance_amount
                   FROM transactions
                   WHERE Value_date IN (
                       SELECT MAX(Value_date)
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                       GROUP BY YEAR(Value_date), QUARTER(Value_date)
                   )
                   GROUP BY YEAR(Value_date), QUARTER(Value_date)"""
},
    {
    'Question': "Give me the breakdown of my income of each month this year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the breakdown of my savings of each month this year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the breakdown of my expenses of each month last year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the breakdown of my income of each month last year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the breakdown of my savings of each month last year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the breakdown of my expenses of each month in year 2022?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE YEAR(Value_date) = 2022
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the breakdown of my income of each month in year 2022?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = 2022
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the breakdown of my savings of each month in year 2022?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = 2022
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the breakdown of my expenses of each month in the last 6 months?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the breakdown of my income of each month in the last 6 months?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
}
    ,
    {
    'Question': "Provide a detailed breakdown of my income by month this year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show me a detailed breakdown of my savings for each month this year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Display an itemized summary of my expenses by month last year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Present a detailed breakdown of my income by month last year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Provide an overview of my savings for each month last year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show a detailed breakdown of my expenses by month in year 2022.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE YEAR(Value_date) = 2022
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Display an itemized summary of my income by month in year 2022.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = 2022
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Provide a breakdown of my savings for each month in year 2022.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = 2022
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show an itemized summary of my expenses by month in the last 6 months.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Present a detailed summary of my income by month in the last 6 months.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
}
,
    {
    'Question': "Give me the detailed breakdown of my expenditures for each month this year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show me an itemized summary of my income by month last year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What were my total expenses each month in the previous year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Provide a breakdown of my income by month in the last 12 months.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show a detailed summary of my savings by month last year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Display an itemized overview of my expenses by month in year 2023.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE YEAR(Value_date) = 2023
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Provide a detailed breakdown of my income by month in year 2023.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = 2023
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show an itemized summary of my savings by month in the last 6 months.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What were my total expenditures each month in the last 6 months?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the detailed breakdown of my income by month in the previous year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
}
,
    {
    'Question': "Show me the breakdown of my expenditures for each month this year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What was my total income by month in the previous year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Provide the expenses for each month in the last 12 months.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me a detailed breakdown of my savings by month last year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show the overview of my expenses by month in the year 2023.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE YEAR(Value_date) = 2023
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Display my income details by month in the year 2023.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = 2023
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Provide a summary of my savings by month in the last 6 months.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What were my total expenditures each month in the last 6 months?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the income breakdown by month in the previous year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show me the detailed expenses by month for the current year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
}
,
    {
    'Question': "What is the average monthly income for each year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, AVG(Deposit_amount) AS Average_Income
                   FROM transactions
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the breakdown of my expenditures by quarter in the current year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, QUARTER(Value_date) AS Quarter, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), QUARTER(Value_date)"""
},
{
    'Question': "Show the trend of my savings over the last 12 months.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What was my total income by month for the previous year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the average monthly expenses for each quarter this year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, QUARTER(Value_date) AS Quarter, AVG(Withdrawal_amount) AS Average_Expenses
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), QUARTER(Value_date)"""
},
{
    'Question': "What are my total savings by month in the current year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show me the breakdown of my monthly income in the previous year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What were my total expenditures by month in the previous year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the average monthly savings for each quarter last year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, QUARTER(Value_date) AS Quarter, AVG(Deposit_amount - Withdrawal_amount) AS Average_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), QUARTER(Value_date)"""
},
{
    'Question': "Show the trend of my expenses over the last 6 months.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
}

,
    {
    'Question': "What is my total income for each month in the current year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show me the trend of my balance over the last 6 months.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, Balance_amount
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What is my average monthly spending for each quarter in this year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, QUARTER(Value_date) AS Quarter, AVG(Withdrawal_amount) AS Average_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), QUARTER(Value_date)"""
},
{
    'Question': "Give me the breakdown of my savings by month for the current year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show me the breakdown of my monthly income in the previous year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What were my total expenditures by month in the previous year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the average monthly savings for each quarter last year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, QUARTER(Value_date) AS Quarter, AVG(Deposit_amount - Withdrawal_amount) AS Average_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), QUARTER(Value_date)"""
},
{
    'Question': "Show the trend of my expenses over the last 6 months.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What is my total income by month for the previous year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Income
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
}
,
    {
    'Question': "What are my monthly expenditures so far this year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "How much have I spent monthly this year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Tell me my monthly spending breakdown for this year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What are my expenses for each month in this calendar year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me a monthly breakdown of my spending this year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Show my monthly expenses throughout this year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What are my expenditures for each month in the current year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Give me the detailed breakdown of my spending by month in this year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "What is the monthly summary of my expenses this year?",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
{
    'Question': "Provide the breakdown of my monthly spending so far this year.",
    'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY YEAR(Value_date), MONTH(Value_date)"""
},
    {
        "Question": "List my top 5 largest deposits of all time.",
        "SQLQuery": "SELECT Transaction_details, Deposit_amount FROM transactions WHERE Deposit_amount > 0 ORDER BY Deposit_amount DESC LIMIT 5"
    },
    {
        "Question": "Show me the highest withdrawal amounts this year.",
        "SQLQuery": "SELECT YEAR(Value_date) AS Year, MAX(Withdrawal_amount) AS Highest_Withdrawal FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())"
    },
    {
        "Question": "What are my top 10 largest transactions by amount?",
        "SQLQuery": "SELECT Transaction_details, (Deposit_amount - Withdrawal_amount) AS Transaction_amount FROM transactions ORDER BY Transaction_amount DESC LIMIT 10"
    },
    {
        "Question": "List my transactions ordered by date.",
        "SQLQuery": "SELECT Transaction_details, Value_date FROM transactions ORDER BY Value_date"
    },
    {
        "Question": "Show me the total deposits grouped by month this year.",
        "SQLQuery": "SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Deposits FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY YEAR(Value_date), MONTH(Value_date)"
    },
    {
        "Question": "Show me the largest withdrawals by day this week.",
        "SQLQuery": "SELECT DATE(Value_date) AS Date, MAX(Withdrawal_amount) AS Largest_Withdrawal FROM transactions WHERE WEEK(Value_date) = WEEK(CURRENT_DATE()) GROUP BY DATE(Value_date)"
    },
    {
        "Question": "What are my top 5 largest deposits ordered by date?",
        "SQLQuery": "SELECT Transaction_details, Deposit_amount, Value_date FROM transactions WHERE Deposit_amount > 0 ORDER BY Value_date DESC LIMIT 5"
    }
,

    {
        "Question": "What are my top 5 largest withdrawals this year?",
        "SQLQuery": "SELECT Transaction_details, Withdrawal_amount FROM transactions WHERE Withdrawal_amount > 0 AND YEAR(Value_date) = YEAR(CURRENT_DATE()) ORDER BY Withdrawal_amount DESC LIMIT 5"
    },
    {
        "Question": "Show me the average deposit amount per month this year.",
        "SQLQuery": "SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, AVG(Deposit_amount) AS Average_Deposit FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY YEAR(Value_date), MONTH(Value_date)"
    },
    {
        "Question": "What are my top 10 largest deposits by amount?",
        "SQLQuery": "SELECT Transaction_details, Deposit_amount FROM transactions WHERE Deposit_amount > 0 ORDER BY Deposit_amount DESC LIMIT 10"
    },

    {
        "Question": "Show me the largest withdrawals by month this year.",
        "SQLQuery": "SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, MAX(Withdrawal_amount) AS Largest_Withdrawal FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY YEAR(Value_date), MONTH(Value_date)"
    },
    {
        "Question": "What are my top 5 largest transactions by amount ordered by date?",
        "SQLQuery": "SELECT Transaction_details, (Deposit_amount - Withdrawal_amount) AS Transaction_amount, Value_date FROM transactions ORDER BY Transaction_amount DESC, Value_date LIMIT 5"
    },
    {
        "Question": "Provide a breakdown of my expenses ordered by transaction details this year.",
        "SQLQuery": "SELECT Transaction_details, SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Withdrawal_amount > 0 AND YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY Transaction_details ORDER BY Total_Expenses DESC"
    },
    {
        "Question": "List my transactions grouped by account number and ordered by the highest withdrawal amount.",
        "SQLQuery": "SELECT Account_No, Transaction_details, MAX(Withdrawal_amount) AS Highest_Withdrawal FROM transactions GROUP BY Account_No, Transaction_details ORDER BY Highest_Withdrawal DESC"
    },
    {
        "Question": "Show me the transactions with the lowest balance amount by month.",
        "SQLQuery": "SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, Transaction_details, MIN(Balance_amount) AS Lowest_Balance FROM transactions GROUP BY YEAR(Value_date), MONTH(Value_date), Transaction_details ORDER BY Lowest_Balance"
    }
    ,
    {
    "Question": "What is my average spendings of last 11 transactions?",
    "SQLQuery": "SELECT AVG(Withdrawal_amount) AS Average_Spending FROM (SELECT Withdrawal_amount FROM transactions WHERE Withdrawal_amount IS NOT NULL ORDER BY Value_date DESC LIMIT 11) AS Last11Transactions"
},
{
    "Question": "What is my average spendings of last 7 transactions?",
    "SQLQuery": "SELECT AVG(Withdrawal_amount) AS Average_Spending FROM (SELECT Withdrawal_amount FROM transactions WHERE Withdrawal_amount IS NOT NULL ORDER BY Value_date DESC LIMIT 7) AS Last7Transactions"
},
{
    "Question": "What is my spendings of last 9 transactions?",
    "SQLQuery": "SELECT SUM(Withdrawal_amount) AS Total_Spending FROM (SELECT Withdrawal_amount FROM transactions WHERE Withdrawal_amount IS NOT NULL ORDER BY Value_date DESC LIMIT 9) AS Last9Transactions"
},
{
    "Question": "What is my spendings of last 4 transactions?",
    "SQLQuery": "SELECT SUM(Withdrawal_amount) AS Total_Spending FROM (SELECT Withdrawal_amount FROM transactions WHERE Withdrawal_amount IS NOT NULL ORDER BY Value_date DESC LIMIT 4) AS Last4Transactions"
},
    {
    "Question": "What is my net worth this year for account number xa1 ?",
    "SQLQuery": "SELECT Balance_amount AS Net_Worth FROM transactions WHERE Account_No = 'xa1' AND YEAR(Value_date) = YEAR(CURRENT_DATE) ORDER BY Value_date DESC LIMIT 1"
},
    {
    "Question": "In which date did i make my highest transaction of this month and what was the amount?",
    "SQLQuery": "SELECT Value_date, MAX(Withdrawal_amount) AS Highest_Transaction_Amount FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH) GROUP BY Value_date ORDER BY Highest_Transaction_Amount DESC LIMIT 1"
},
    {
        "Question": "In which date did i make my highest transaction of this week and what was the amount?",
        "SQLQuery": "SELECT Value_date, MAX(Withdrawal_amount) AS Highest_Transaction_Amount FROM transactions WHERE Value_date >= DATE_SUB(CURDATE(), INTERVAL 1 WEEK) GROUP BY Value_date ORDER BY Highest_Transaction_Amount DESC LIMIT 1"
    },
    {
        "Question": "What is my highest transaction amount of this year and what was the date?",
        "SQLQuery": "SELECT Value_date, MAX(Withdrawal_amount) AS Highest_Transaction_Amount FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY Value_date ORDER BY Highest_Transaction_Amount DESC LIMIT 1"
    },
    {
        "Question": "What is my highest transaction amount of previous year and what was the date ? ",
        "SQLQuery": "SELECT Value_date, MAX(Withdrawal_amount) AS Highest_Transaction_Amount FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1 GROUP BY Value_date ORDER BY Highest_Transaction_Amount DESC LIMIT 1"
    }
,
    {
        "Question": "What is the lowest transaction amount of this month and on which date?",
        "SQLQuery": "SELECT Value_date, MIN(Withdrawal_amount) AS Lowest_Transaction_Amount FROM transactions WHERE MONTH(Value_date) = MONTH(CURRENT_DATE()) GROUP BY Value_date ORDER BY Lowest_Transaction_Amount LIMIT 1"
    },
    {
        "Question": "In which date did I make my lowest transaction of this year and what was the amount?",
        "SQLQuery": "SELECT Value_date, MIN(Withdrawal_amount) AS Lowest_Transaction_Amount FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY Value_date ORDER BY Lowest_Transaction_Amount LIMIT 1"
    },
    {
        "Question": "What is my highest deposit amount of last month and what was the date?",
        "SQLQuery": "SELECT Value_date, MAX(Deposit_amount) AS Highest_Deposit_Amount FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH) GROUP BY Value_date ORDER BY Highest_Deposit_Amount DESC LIMIT 1"
    },
    {
        "Question": "What is the lowest deposit amount of last year and what was the date?",
        "SQLQuery": "SELECT Value_date, MIN(Deposit_amount) AS Lowest_Deposit_Amount FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1 GROUP BY Value_date ORDER BY Lowest_Deposit_Amount LIMIT 1"
    },

    {
        "Question": "What is the highest balance amount recorded this month and on which date?",
        "SQLQuery": "SELECT Value_date, MAX(Balance_amount) AS Highest_Balance FROM transactions WHERE  year(Value_date) = year(CURRENT_DATE()) And MONTH(Value_date) = MONTH(CURRENT_DATE()) GROUP BY Value_date ORDER BY Highest_Balance DESC LIMIT 1"

    },
    {
        "Question": "What is the lowest balance amount recorded last year and on which date?",
        "SQLQuery": "SELECT Value_date, MIN(Balance_amount) AS Lowest_Balance FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1 GROUP BY Value_date ORDER BY Lowest_Balance LIMIT 1"
    },
    {
        "Question": "List the top 5 highest withdrawal transactions of this year, including their dates and amounts.",
        "SQLQuery": "SELECT Value_date, Transaction_details, Withdrawal_amount FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) ORDER BY Withdrawal_amount DESC LIMIT 5"
    },
    {
        "Question": "List the top 10 highest deposit transactions of last month, including their dates and amounts.",
        "SQLQuery": "SELECT Value_date, Transaction_details, Deposit_amount FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH) ORDER BY Deposit_amount DESC LIMIT 10"
    }
    ,{
    "Question": "How much amount did I spend each month last year?",
    "SQLQuery": "SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE YEAR(Value_date) = YEAR(CURDATE()) - 1 GROUP BY YEAR(Value_date), MONTH(Value_date) ORDER BY Year DESC, Month DESC;"
}
  ,
    {
        'Question': "How much amount did I earned in the 3rd quarter of this year ?",
        'SQLQuery': """SELECT SUM(Deposit_amount) AS Earnings_3rd_Quarter
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                             AND QUARTER(Value_date) = 3"""
    },
    {
        'Question': "How much amount did I earned in the 1st quarter of this year ?",
        'SQLQuery': """SELECT SUM(Deposit_amount) AS Earnings_1st_Quarter
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                             AND QUARTER(Value_date) = 1"""
    },
    {
        'Question': "How much amount did I spent in the 2nd quarter of this year ?",
        'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Spending_2nd_Quarter
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                             AND QUARTER(Value_date) = 2"""
    },
    {
        'Question': "How much amount did I earned in June month ?",
        'SQLQuery': """SELECT SUM(Deposit_amount) AS Earnings_June
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                             AND MONTH(Value_date) = 6"""
    },
    {
        'Question': "How much amount did I earned in October month ?",
        'SQLQuery': """SELECT SUM(Deposit_amount) AS Earnings_October
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                             AND MONTH(Value_date) = 10"""
    },
    {
        'Question': "How much amount did I spent on February month ?",
        'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Spending_February
                       FROM transactions
                       WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                             AND MONTH(Value_date) = 2"""
    },
    {
        'Question': "How much I earned in total last month?",
        'SQLQuery': """
            SELECT SUM(Deposit_amount) AS Total_Earned_Last_Month
      FROM transactions where month(Value_date) = month(current_date()-interval 1 month)
      and year(value_date)=Year(current_date());
        """
    },
    {
        'Question': "How many fund transfers I did in last 30 days?",
        'SQLQuery': """
            SELECT COUNT(*) AS Fund_Transfers_Last_30_Days
            FROM transactions
            WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY);
        """
    },
     {
        'Question': "How many fund transfers I did in last 70 days?",
        'SQLQuery': """
            SELECT COUNT(*) AS Fund_Transfers_Last_70_Days
            FROM transactions
            WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 70 DAY);
        """
    },
    {
        'Question': "What is my consistent income in my last 3 months?",
        'SQLQuery': """
          SELECT AVG(Deposit_amount) AS Average_Monthly_Income
            FROM transactions
            WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH);
        """
    },
    {
        'Question': "What is my consistent income in my last 8 months?",
        'SQLQuery': """
          SELECT AVG(Deposit_amount) AS Average_Monthly_Income
            FROM transactions
            WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 8 MONTH);
        """
    },
    {
        'Question': "What are my total expenses for the last three months?",
        'SQLQuery': """
            SELECT SUM(Withdrawal_amount) AS Total_Expenses_Last_3_Months
            FROM transactions
            WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH)
        """
    },
    {
        'Question': "How much have I spent on groceries this month?",
        'SQLQuery': """
            SELECT SUM(Withdrawal_amount) AS Total_Grocery_Expenses_This_Month
            FROM transactions
            WHERE Value_date >= DATE_SUB(LAST_DAY(CURRENT_DATE() - INTERVAL 0 MONTH), INTERVAL 0 MONTH)
            AND Transaction_details LIKE '%GROCERIES%'
        """
    },
    {
        'Question': "What was my total spending last week?",
        'SQLQuery': """
            SELECT SUM(Withdrawal_amount) AS Total_Spending_Last_Week
            FROM transactions
            WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 WEEK)
        """
    },
    {
        'Question': "Can you give me a breakdown of my expenses for each day this week?",
        'SQLQuery': """
         SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 WEEK) GROUP BY DATE(Value_date);

        """
    },
    {
        'Question': "How many transactions did I make last month?",
        'SQLQuery': """
            SELECT COUNT(*) AS Total_Transactions_Last_Month
            FROM transactions
            WHERE YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 MONTH)
            AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH)
        """
    },
    {
        'Question': "How many transactions were made for entertainment in April?",
        'SQLQuery': """
            SELECT COUNT(*) AS Entertainment_Transactions_April
            FROM transactions
            WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
            AND MONTH(Value_date) = 4
            AND Transaction_details LIKE '%ENTERTAINMENT%'
        """
    },
     {
        'Question': "What was my total income last month?",
        'SQLQuery': """
              SELECT year(Value_date) as year,month(value_date) as month, SUM(Deposit_amount) AS Total_Income_Last_Month
            FROM transactions
            WHERE YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 MONTH)
            AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH)
            group by year(Value_date),month(value_date);
        """
    },
    {
        'Question': "How much did I save last month?",
        'SQLQuery': """
            SELECT SUM(Deposit_amount - Withdrawal_amount) AS Total_Savings_Last_Month
            FROM transactions
            WHERE YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 MONTH)
            AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH)
        """
    }
 ,
    {
        'Question':" Income of last 3 months. ?",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH) """,
        'SQLResult':"1530205868",
        'Answer':"The income of last 3 months is 1530205868 ."
    }
,
    {
        'Question':"How many transactions did I make in last 2 week as my account number is 409000493201? ",
        'SQLQuery':"""SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 WEEK) AND ACCOUNT_NO= '409000493201' """,
        'SQLResult':"162",
        'Answer':"You made 162 transactions in last 2 week."
    }
    ,
     {
        'Question':"Amount spent last week as my account number is 409000493201?",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE year(value_date) = year(curdate()) and week(value_date)=week(curdate())-1  AND ACCOUNT_NO= '409000493201' """,
        'SQLResult':"626764",
        'Answer':"You spent total RS. 626764 last week ."
    }
    , {
        'Question': "Expenses for each day this month ?",
        'SQLQuery': """SELECT value_date, SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) GROUP BY value_date """,
        'SQLResult':
        """
[(2024, 6, 2, Decimal('183648')),
(2024, 6, 3, Decimal('80666')),
(2024, 6, 4, Decimal('135504')),
(2024, 6, 5, Decimal('92031')),
(2024, 6, 6, Decimal('118475')),
(2024, 6, 7, Decimal('16440')),
(2024, 6, 9, Decimal('62112')),
(2024, 6, 10, Decimal('10331'))]
"""
        ,
        'Answer': """The total_expenses for each day of this month are:
        2024 June, 2:RS. 183648,
        2024 June, 3:RS. 80666,
        2024 June, 4:RS. 135504,
        2024 June, 5:RS. 92031,
        2024 June, 6:RS. 118475,
        2024 June, 7:RS. 16440,
        2024 June, 9:RS. 62112,
        2024 June, 10:RS. 10331
        """
    },
    {
        'Question':"savings of previous year for each month ",
        'SQLQuery':"""SELECT year(Value_date), MONTH(Value_date) AS Month, (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Total_Savings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 YEAR) GROUP BY year(value_date), MONTH(Value_date) ORDER BY MONTH(Value_date); """,
        'SQLResult':"""
       [(2023, 1, Decimal('-311244')),
        (2023, 2, Decimal('9101')),
        (2023, 3, Decimal('3476')),
        (2023, 4, Decimal('-15971')),
        (2023, 5, Decimal('322249')),
        (2023, 6, Decimal('-172560')),
        (2023, 7, Decimal('361471')),
        (2023, 8, Decimal('-446844')),
        (2023, 9, Decimal('93215')),
        (2023, 10, Decimal('-73917')),
        (2023, 11, Decimal('294502')),
        (2023, 12, Decimal('-404028'))]
        """,
        'Answer':"""
        Your Savings for each month of previous year are:
        January:RS. -311244, February:RS. 9101, March:RS. 3476
        April:RS. -15971, May:RS. 322249, June:RS. -172560
        July:RS. 361471, August:RS. -446844, September:RS. 93215
        October:RS. -73917, November:RS. 294502, December:RS. -404028 """
    },
     {
        'Question':"Amount Saved each year",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year,SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions GROUP BY YEAR(Value_date) """,
        'SQLResult':"""
      [(2021, Decimal('497766')),
        (2022, Decimal('272458')),
        (2023, Decimal('-340550')),
        (2024, Decimal('-148114'))]
       """,
        'Answer':"""You saved following amount each year:
                2021:RS. 497766, 2022:RS. 272458, 2023:RS. -340550
                2024:RS. -148114 """
    }
    ,
     {
        'Question':"Savings of last 2 year as my account number is 409000493201? ",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Net_Savings FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 YEAR) AND ACCOUNT_NO= '409000493201' GROUP BY YEAR(Value_date) """,
        'SQLResult':"""
       [(2022, Decimal('2270453')),
    (2023, Decimal('-895783')),
    (2024, Decimal('-912470'))]

        """,
        'Answer':"""Your savings of last 2 year are:
         2022:RS. 2270453, 2023:RS.-895783
                2024:RS. -912470 ."""
    }

    #&&&&&&&&&&&&&&&&&&&&
    ,
     {
        'Question':"Expenses of each day of last week for account number 409000438611  ",
        'SQLQuery':"""SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE  YEAR(Value_date) = YEAR(CURDATE()) AND week(Value_date) = week(CURDATE())-1 and account_no='409000438611' GROUP BY DATE(Value_date)""",
        'SQLResult':"""
       [(2024-07-14, Decimal('12500000')),
    (2024-07-15, Decimal('4900000')),
    (2024-07-16, Decimal('3901373')),
    (2024-07-17, Decimal('4400000')),
    (2024-07-18, Decimal('4800000')),
    (2024-07-19, Decimal('2700000'))]
        """,
        'Answer':"""You spend following amount each day last week:
        July 14:RS. 12500000, July 15:RS. 3901373, July 16:RS. 3901373, July 17:RS. 4400000, July 18:RS. 4800000, July 19:RS. 2700000 """
    }
    ,
    {
        'Question':"Amount spent this year as my account number is 409000493201??",
        'SQLQuery':"""SELECT year(Value_date) as Year,sum(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE year(Value_date) = year(curdate()) AND ACCOUNT_NO= '409000493201' group by year(Value_date) """,
        'SQLResult':"""[(2024, Decimal('6527128485'))]""",
        'Answer':"You spent RS. 6527128485 on 2024"
    },
    {
    'Question': "What is my consistent income in last 2 months as my account number is 409000493201  ?",
    'SQLQuery': """SELECT AVG(Monthly_Income) AS Consistent_Income
    FROM (SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Monthly_Income FROM transactions WHERE  Value_date >=
    DATE_SUB(CURRENT_DATE(), INTERVAL 2 MONTH) AND ACCOUNT_NO= '409000493201'
    GROUP BY  YEAR(Value_date), MONTH(Value_date)) AS Income_Last_3_Months;
    """,
    'SQLResult': "2466666.6667",
    'Answer': "The consistent income of last 2 months is RS. 2466666.6667"
}
    ,
    {
        'Question':"amount/money saved each month previous year ?",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions WHERE YEAR(Value_date) = YEAR(DATE_SUB(CURRENT_DATE(), INTERVAL 1 YEAR)) GROUP BY YEAR(Value_date), MONTH(Value_date) """,
        'SQLResult':"""
[(2023, 1, Decimal('-311244')),
(2023, 2, Decimal('9101')),
(2023, 3, Decimal('3476')),
(2023, 4, Decimal('-15971')),
(2023, 5, Decimal('322249')),
(2023, 6, Decimal('-172560')),
(2023, 7, Decimal('361471')),
(2023, 8, Decimal('-446844')),
(2023, 9, Decimal('93215')),
(2023, 10, Decimal('-73917')),
(2023, 11, Decimal('294502')),
(2023, 12, Decimal('-404028'))]
        """,
        'Answer':"""You saved following amounts previous year :
        January: -311244, February: 9101, March: 3476, April: -15971, May: 322249, June: 172560, July: 361471,
        August: 446844, September: 93215, October: 73917, November: 294502, December: 404028 """
    }
    ,
{
    'Question': "What is my total income last quarter?",
    'SQLQuery': """SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND QUARTER(Value_date) = QUARTER(CURRENT_DATE()) - 1""",
    'SQLResult':"2055417596",
    'Answer':"Your total income last quater was RS. 2055417596."
}
     ,
      {
        'Question':"income of the previous year ",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1 """,
        'SQLResult':"38593214103",
        'Answer':"Your income previous ysear was 38593214103."
    }
    ,
      {
        'Question':"income of the first 4 month of previous year ",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE MONTH(Value_date) IN (1,2,3,4) AND YEAR(Value_date) = YEAR(CURRENT_DATE) - 1 """,
        'SQLResult':"17847505444",
        'Answer':"Your income of first 4 month of previous year was 17847505444."
    }
    ,
    {
    'Question': "amount I lost each year?",
    'SQLQuery': """ SELECT YEAR(Value_date) AS Year,SUM(Withdrawal_amount) - SUM(Deposit_amount) AS Losses FROM transactions GROUP BY YEAR(Value_date) """,
    'SQLResult': """[
        (2022, Decimal('256395842')),
        (2023, Decimal('552520828')),
        (2024, Decimal('505236609')),
        (2021, Decimal('1251859488')),
        (2020, Decimal('694854538'))]"""
    ,
    'Answer': "You lost RS. 694854538 in 2020, RS. 1251859488 in 2021, RS. 256395842 in 2022,RS. 552520828 in 2023, and RS. 505236609 in 2024."
}
,
{
    "Question": "What is my average spendings of last 11 transactions?",
    "SQLQuery": """SELECT AVG(Withdrawal_amount) AS Average_Spending FROM (SELECT Withdrawal_amount FROM transactions WHERE Withdrawal_amount
                IS NOT NULL ORDER BY Value_date DESC LIMIT 11) AS Last11Transactions""",
    "SQLResult":"1138377.5455",
    'Answer': "1138377.5455 is your average spendings of last 11 transactions ."
}
,
{
    'Question': "How much did I spend each quarter this year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY QUARTER(Value_date)""",
    'SQLResult':"""[(1, Decimal('5885524375')),
                    (2, Decimal('2689194690')),
                    (3, Decimal('263256582'))]""",
    'Answer': """Spendings for each quater is :
                    Quarter-1:RS. 5885524375, Quarter-2:RS. 2689194690, Quarter-3:RS. 263256582"""
},

{
    'Question': "In which date did i make my highest transaction of this month and what was the amount?",
    'SQLQuery': "SELECT Value_date, MAX(Withdrawal_amount) AS Highest_Transaction_Amount FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH) GROUP BY Value_date ORDER BY Highest_Transaction_Amount DESC LIMIT 1",
    'SQLResult': "[(2024-06-25, Decimal('10000000'))]",
    'Answer': "You made the highest transaction on Junt 25th and the amount is RS. 10000000"
}
,
 {
        'Question': "How much amount did I spent on February month ?",
        'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Spending_February FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = 2""",
        'SQLResult': "1985783353",
        'Answer': "You spent RS. 1985783353 on February month."
    },
    {
        'Question': "What were my total expenditures by month in the previous year?",
        'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1 GROUP BY YEAR(Value_date), MONTH(Value_date)""",
        'SQLResult':"""[(2023, 1, Decimal('4530730974')), (2023, 2, Decimal('4616646063')), (2023, 3, Decimal('5276664685')), (2023, 4, Decimal('4571039249')), (2023, 5, Decimal('3557364039')), (2023, 6, Decimal('3262112460')), (2023, 7, Decimal('3142750834')), (2023, 8, Decimal('2875423837')), (2023, 9, Decimal('2727745395')), (2023, 10, Decimal('2537575257')), (2023, 11, Decimal('2818821756')), (2023, 12, Decimal('2246381750'))]""",
        'Answer': """Expenditure for each month previous year:
            January:RS. 4530730974, February: RS. 4616646063, March: RS. 5276664685, April: RS. 4571039249, May:RS. 3557364039, June: RS. 3262112460, July:RS. 3142750834, August:RS. 2875423837, September:RS. 2727745395, October:RS. 2537575257, November:RS. 2818821756, December:RS. 2246381750 """
}

,{
        'Question': "Which is my highest earning month this year ? I also want the amount along with month",
        'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Earnings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY MONTH(Value_date) ORDER BY Total_Earnings DESC limit 1""",
    'SQLResult': "[(1, Decimal('1893013978'))]",
        'Answer': "Your highest earning month this year is June with amount 1893013978."
    }
    ,{
        'Question': "What was my total spending last week? as my account number is 409000438611 ",
        'SQLQuery': """SELECT sum(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE  YEAR(Value_date) = YEAR(CURDATE()) AND week(Value_date) = week(CURDATE())-1
        AND ACCOUNT_NO= '409000438611'""",
        'SQLResult': "33201373",
        'Answer': "Your total expenses last week was RS. 33201373."
    },
    {
    "Question": "Can you give me a breakdown of my expenses for each day this week?",
    "SQLQuery": "SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Expenses FROM transactions WHERE YEAR(Value_date) = YEAR(CURDATE()) AND WEEK(Value_date) = WEEK(CURDATE()) GROUP BY DATE(Value_date) ORDER BY Date ASC",
    'SQLResult':"""(2024-07-21, Decimal('9967448')),
        (2024-07-22, Decimal(32142360'))""",
    'Answer': "Daily expenses for the week: July 21st: RS. 9967448, July 22nd: RS. 32142360."
},
{
        'Question': "How much did i save last month as my account number is 409000493201 ",
        'SQLQuery': """  SELECT  SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Total_Savings
FROM transactions WHERE year(value_date) = year(curdate()) and month(value_date)=month(curdate()) -1 AND Account_No = '409000493201';
""",
    'SQLResult': "89804",
        'Answer': "Your savings last month was RS. 89804."
    }
    ,
    {
        'Question': "What is my total income of last month as my account number is 409000425051 ",
        'SQLQuery': """  SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE year(value_date) = year(curdate()) and month(value_date) = month(curdate()) -1 AND Account_No = '409000425051' """,
    'SQLResult': "5900000",
        'Answer': "Your total income last month is RS. 5900000."
    },
    {
    'Question': "Breakdown my expenses for each week for the last month as my account number is 409000493201.",
    'SQLQuery': """SELECT WEEK(Value_date) AS Week, SUM(Withdrawal_amount) AS Weekly_Expenses
                   FROM transactions
                   WHERE Account_No = '409000493201'
                   AND Value_date >= DATE_SUB(LAST_DAY(DATE_SUB(CURDATE(), INTERVAL 1 MONTH)), INTERVAL DAY(LAST_DAY(DATE_SUB(CURDATE(), INTERVAL 1 MONTH)))-1 DAY)
                   AND Value_date <= LAST_DAY(DATE_SUB(CURDATE(), INTERVAL 1 MONTH))
                   GROUP BY 1;""",
    'SQLResult': """[(22, Decimal('1113224')),
                    (23, Decimal('1030576')),
                    (23, Decimal('1394682')),
                    (24, Decimal('847515')),
                    (25, Decimal('224199'))]""",
    'Answer': """ Your expenses were:
    1st week : RS. 1113224,
    2nd week :RS. 1030576,
    3rd week: RS. 1394682,
    4th week:RS. 847515 ,
    5th week:RS. 224199"""

}
,
{
        'Question': "What is my total income for this year so far as my account number is 409000611074?",
        'SQLQuery': """SELECT SUM(Deposit_amount) AS Total_Income
FROM transactions
WHERE YEAR(Value_date) = YEAR(CURDATE())
AND Account_No = '409000611074';
 """,
    'SQLResult': "59000000",
        'Answer': "The total income this year is RS. 59000000 "
    },
    {
        'Question': "How much did I spend in the last quarter of the previous year? as my account number is 409000611074?",
        'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Total_Spending
FROM transactions
WHERE YEAR(Value_date) = YEAR(CURDATE()) - 1
AND Account_No = '409000611074'
AND QUARTER(Value_date) = 4; """,
    'SQLResult': "32120960",
        'Answer': "Your spendings of last quater is RS.32120960 "
    },
    {
        'Question':  "Can you give me a monthly breakdown of my income for the past 6 months? as my account number is 409000493201",
        'SQLQuery': """ SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Monthly_Income
FROM transactions
WHERE Value_date >= DATE_SUB(CURDATE(), INTERVAL 6 MONTH)
AND Account_No = '409000493201'
GROUP BY YEAR(Value_date), MONTH(Value_date);""",
    'SQLResult': """[(2024,1, Decimal('715160')),
                    (2024,2, Decimal('4584217')),
                    (2024,3, Decimal('9112141')),
                    (2024,4, Decimal('11443808')),
                    (2024,5, Decimal('5411781')),
                    (2024,6, Decimal('4700000')),
                    (2024,7, Decimal('2100000'))]""",
        'Answer': """Here is the breakdown of your monthly income for past 6 months:
        January: RS. 715160,
        February: RS. 4584217,
        March: RS. 9112141,
        April: RS. 11443808,
        May: RS. 5411781,
        June: RS. 4700000,
        July: RS. 2100000
        """
    },
    {
        'Question': "What is the total amount of money deposited over the past year for account number 409000493201?",
        'SQLQuery': """ SELECT SUM(Deposit_amount) AS Total_Deposits
FROM transactions
WHERE YEAR(Value_date) = YEAR(CURDATE()) - 1
AND Account_No = '409000493201'""",
    'SQLResult': "39109096",
        'Answer': "The total amount of money deposited over the past year is Rs. 39109096"
    },
    #$$$$$$

{
    'Question': "What were my total losses for each month last year for account number 409000493201?",
    'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Withdrawal_amount) - SUM(Deposit_amount) AS Monthly_Losses
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURDATE()) - 1
                   AND Account_No = '409000493201'
                   GROUP BY MONTH(Value_date);""",
    'SQLResult': """[(1, Decimal('463794')),
                    (2, Decimal('-5936')),
                    (3, Decimal('-262405')),
                    (4, Decimal('403300')),
                    (5, Decimal('-170150')),
                    (6, Decimal('-363342')),
                    (7, Decimal('456354')),
                    (8, Decimal('-486051')),
                    (9, Decimal('267825')),
                    (10, Decimal('-179758')),
                    (11, Decimal('74787')),
                    (12, Decimal('-122188'))]""",
    'Answer': """Your loss for each month last year is:
    January: RS. 463794,
     February:RS.-5936,
     March:RS. -262405,
     April:RS. 403300,
     May:RS. -170150,
     June:RS. -363342,
     July:RS. 456354,
     August:RS. -486051,
     September:RS. 267825,
     October:RS. -179758,
     November:RS. 74787,
     December:RS. -122188"""
},
{
    'Question': "What is the highest single deposit amount made this year for account number 409000493201?",
    'SQLQuery': """SELECT MAX(Deposit_amount) AS Highest_Deposit
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURDATE())
                   AND Account_No = '409000493201';""",
    'SQLResult': "615753",
    'Answer': "The highest deposit made this year is RS. 615753"
},
{
    'Question': "How much did I spend on the 1st and 2nd of this month for account number 409000493201?",
    'SQLQuery': """SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURDATE())
                   AND MONTH(Value_date) = MONTH(CURDATE())
                   AND DAY(Value_date) IN (1, 2)
                   AND Account_No = '409000493201'
                   GROUP BY DATE(Value_date);""",
    'SQLResult': """[(2024-07-01, Decimal('116967')),
                    (2024-07-02, Decimal('90749'))""",
    'Answer': """You spent following amount :
    07-01: RS. 116967,
     07-02: RS. 90749 """
},
{
    'Question': "Can you show my total savings for each quarter of the current year for account number 409000493201?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Quarterly_Savings
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURDATE())
                   AND Account_No = '409000493201'
                   GROUP BY QUARTER(Value_date);""",
    'SQLResult': """[(1, Decimal('-566948')),
                    (2, Decimal('-74404')),
                    (3, Decimal('44031'))""",
    'Answer': "1st quater : you lossed RS. -566948 ,2nd quater : you lossed RS. 74404 ,3rd quater : you earned RS 44031"
}
,{
    'Question': "I want to buy a car worth RS. 500000 in next 2 years . So how much should i save each month to buy the car  for account number 409000493201?",
    'SQLQuery': """SELECT AVG(Monthly_Savings) AS Average_Monthly_Savings
FROM (
    SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month,
           SUM(Deposit_amount - Withdrawal_amount) AS Monthly_Savings
    FROM transactions
    WHERE Account_No = '409000493201'
    GROUP BY YEAR(Value_date), MONTH(Value_date)
) AS Monthly_Savings_Calculation;""",
    'SQLResult': """9385.3333 """,
    'Answer': """Your Average monthly saving is RS. 9385.3333 which is not eought to buy a car worth RS. 500000.
     So you much save around 500000 / 24 = RS. 20833  each month """
},
{
    'Question': "What is my average monthly income in first 6 month this year? as my account number is 409000493201. ",
    'SQLQuery': """SELECT AVG(Monthly_Income) AS Average_Income
FROM (
    SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Monthly_Income
    FROM transactions
    WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
      AND MONTH(Value_date) BETWEEN 1 AND 6
      AND Account_No = '409000493201'
    GROUP BY YEAR(Value_date), MONTH(Value_date)
) AS Income_First_6_Months;
""",
    'SQLResult': """6352764.1667 """,
    'Answer': """Your Average monthly income of the first 6 month this year is RS. 6352764.1667"""
    },
    {
    "Question": "How many transactions did I make last quarter ? as my account number is 409000493201.",
    "SQLQuery": "SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND QUARTER(Value_date) = QUARTER(CURRENT_DATE()) - 1 AND Account_No = '409000493201';",
    'SQLResult': """145""",
    'Answer': """You made 145 transactions last quater."""
},
    {
    'Question': "What is my total number of transactions each month this year?",
    'SQLQuery':""" SELECT MONTH(Value_date) AS Month, COUNT(*) AS Total_Transactions
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY MONTH(Value_date);""",
    'SQLResult': """[(1, 3567),
 (2, 3468),
 (3, 2752),
 (4, 2729),
 (5, 2603),
 (6, 2653),
 (7, 1923)]""",
'Answer':""" You made following number of transactions each month this year:
January: 3567, February: 3468, March: 2752, April: 2729, May: 2603, June: 2653, July: 1923"""
}
,
{
    "Question": "What is my net worth as of today?",
    "SQLQuery": "SELECT Balance_amount AS Net_Worth FROM transactions WHERE Account_No = '409000493201' ORDER BY Value_date DESC LIMIT 1;",
    'SQLResult': """743583 """,
    'Answer': """Your net worth as of today is RS. 743583"""
},
{
    'Question': "How many withdrawals did I make this year?",
    'SQLQuery': """SELECT COUNT(*) AS Total_Withdrawals
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   AND Withdrawal_amount IS NOT NULL""",
    'SQLResult': """19695 """,
    'Answer': """Your made 19695 transaction this year."""
},
{
    'Question': "What is the average daily spending in the last month?",
    'SQLQuery': """SELECT AVG(Daily_Spending) AS Average_Daily_Spending
                   FROM (
                       SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Daily_Spending
                       FROM transactions
                       WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)
                       GROUP BY DATE(Value_date)
                   ) AS DailySpending""",
    'SQLResult': """18662884.0417 """,
    'Answer': """Your average daily spending last month was RS. 18662884.0417"""
},
{
    'Question': "What is my largest withdrawal amount this year? as my account number is 409000425051",
    'SQLQuery': """SELECT MAX(Withdrawal_amount) AS Largest_Withdrawal
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   And Account_no='409000425051'""",
    'SQLResult': """354000000 """,
    'Answer': """Your largest withdrawal amount this year is RS. 354000000"""
},
    {
    'Question': "Can you give me a summary of my financial health as my account number is 1242",
    'SQLQuery': """SELECT
    SUM(Deposit_amount) AS Total_Income,
    SUM(Withdrawal_amount) AS Total_Expenses,
    SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Net_Savings,
    (SELECT Balance_amount FROM transactions WHERE Account_No = '1242' ORDER BY Value_date DESC LIMIT 1) AS Current_Net_Worth
FROM transactions
WHERE Account_No = '1242';"""
},
    {
    'Question': "I want to buy a car worth RS. 5,00,000 in next 2 years . So how much should i save each month to buy the car  for account number 409000493201?",
    'SQLQuery': """SELECT round(AVG(Monthly_Savings),2) AS Average_Monthly_Savings
FROM (
    SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month,
           SUM(Deposit_amount - Withdrawal_amount) AS Monthly_Savings
    FROM transactions
    WHERE Account_No = '409000493201'
    GROUP BY YEAR(Value_date), MONTH(Value_date)
) AS Monthly_Savings_Calculation;""",
    'SQLResult': """9385.33""",
    'Answer': """Lets think step by step:
        step-1) Determine Total Number of Months:
        Time frame: 2 years (12 months in 1 year So 2 years * 12 months = 24) = 24 months
    step-2) Calculate Required Monthly Savings:
        Target amount: RS 500000
        Required monthly savings:(It is obtained by dividing the total amount by Time frame) i.e RS. 500000 / 24 months = RS. 20,833.33
    step-3) Compare Current Savings with Required Savings:
        Current average monthly savings: RS 9385.33
        Difference needed:  RS. 20,833.33 - RS 9385.33 = RS. 11,448
    step-4) Conclusion: To reach the goal of saving RS 5,00,000 in 2 years. increase monthly savings to RS 20,833.33 .The additional amount to save each month is RS. 11,448 """
},
  {
    'Question': "I want to buy a car worth RS 6,00,000 in next 3 years . So how much should i save each month to buy the car ? as my account number is 409000438611. ",
    'SQLQuery': """SELECT AVG(Monthly_Savings) AS Average_Monthly_Savings
FROM (
    SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month,
           SUM(Deposit_amount - Withdrawal_amount) AS Monthly_Savings
    FROM transactions
    WHERE Account_No = '409000438611'
    GROUP BY YEAR(Value_date), MONTH(Value_date)
) AS Monthly_Savings_Calculation;""",
    'SQLResult': """9627.44""",
    'Answer': """Lets think step by step:
        step-1) Determine Total Number of Months:
        Time frame: 3 years (12 months in 1 year So 3 years * 12 months = 36) = 36 months

    step-2) Calculate Required Monthly Savings:
        Target amount: RS 6,00,000
        Required monthly savings:(It is obtained by dividing the total amount by Time frame)RS 600000 / 36 months = RS 16,666.67

    step-3) Compare Current Savings with Required Savings:
        Current average monthly savings: RS 9627.44
        Difference needed: RS 16,666.67 - RS 9,627.44 = RS 7,039.23

    step-4) Conclusion: To reach the goal of saving RS 6,00,000 in 3 years, increase monthly savings to RS 16,666.67 The additional amount to save each month is RS 7,039.23"""
}
,
    {
    'Question': "Show me a summary of my spending for the past month for account number 1281 ",
    'SQLQuery': """ SELECT COUNT(*) AS number_of_transactions,SUM(Withdrawal_amount) AS total_spending,MAX(Withdrawal_amount) AS highest_transaction_amount FROM transactions WHERE year(Value_date) =year(curdate()) And month(value_date) = month(curdate()) - 1 And Account_No = '1281' """
},

    {
    'Question': "Show me a summary of my spending for the past week for account number 409000493201 ",
    'SQLQuery': """ SELECT COUNT(*) AS number_of_transactions,SUM(Withdrawal_amount) AS total_spending,MAX(Withdrawal_amount) AS highest_transaction_amount FROM transactions WHERE year(Value_date) =year(curdate()) And week(value_date) = week(curdate()) - 1 And Account_No = "409000493201" """
},

    {
    'Question': "What is my current net worth for account number 409000493201 ",
    'SQLQuery': """ SELECT Balance_amount AS Net_Worth FROM transactions WHERE Account_No = '409000493201' ORDER BY Value_date DESC LIMIT 1; """
    }
    ,

    {
    'Question': "What is my current net worth for account number 1281 . ",
    'SQLQuery': """ SELECT Balance_amount AS Net_Worth FROM transactions WHERE Account_No = '1281' ORDER BY Value_date DESC LIMIT 1; """
    }
    ,

    {
    'Question': " How much I earned in total last month? ",
    'SQLQuery': """ SELECT SUM(Deposit_amount) AS Total_Earned_Last_Month FROM transactions where month(Value_date) = month(current_date()-interval 1 month) and year(value_date)=Year(current_date()); """
    }
]



In [ ]:
# import pandas as pd

# # Convert the list of dictionaries to a pandas DataFrame
# df = pd.DataFrame(few_shots, columns=['Question', 'SQLQuery'])

# print(df)

In [ ]:
# df.head(10)

In [ ]:
# Database schema and question prefix
import pandas as pd
db_schema = (
    "You are an expert in converting natural anguages into MYSQL queries. Here is the database schema defined by create statement. CREATE TABLE transactions "
    "( `Account_No` VARCHAR(50) NOT NULL, `Transaction_details` TEXT, "
    "`Withdrawal_amount` INTEGER, `Deposit_amount` INTEGER, "
    "`Balance_amount` INTEGER, `Value_date` DATE, `Date` DATE ) . "
    "Please write me a SQL query that answers the given user question:"
)




In [ ]:
# Create the DataFrame
df = pd.DataFrame(few_shots)

# Format the data
df['text'] = df.apply(
    lambda row: f"<s>[INST] <<SYS>> {db_schema} <</SYS>> {row['Question']} [/INST] {row['SQLQuery']} </s>",
    axis=1
)

# Display the DataFrame
print(df[['text']])

                                                  text
0    <s>[INST] <<SYS>> You are an expert in convert...
1    <s>[INST] <<SYS>> You are an expert in convert...
2    <s>[INST] <<SYS>> You are an expert in convert...
3    <s>[INST] <<SYS>> You are an expert in convert...
4    <s>[INST] <<SYS>> You are an expert in convert...
..                                                 ...
333  <s>[INST] <<SYS>> You are an expert in convert...
334  <s>[INST] <<SYS>> You are an expert in convert...
335  <s>[INST] <<SYS>> You are an expert in convert...
336  <s>[INST] <<SYS>> You are an expert in convert...
337  <s>[INST] <<SYS>> You are an expert in convert...

[338 rows x 1 columns]


In [ ]:
df['text'][0]

'<s>[INST] <<SYS>> You are an expert in converting natural anguages into MYSQL queries. Here is the database schema defined by create statement. CREATE TABLE transactions ( `Account_No` VARCHAR(50) NOT NULL, `Transaction_details` TEXT, `Withdrawal_amount` INTEGER, `Deposit_amount` INTEGER, `Balance_amount` INTEGER, `Value_date` DATE, `Date` DATE ) . Please write me a SQL query that answers the given user question: <</SYS>>  Income of last 3 months. ? [/INST] SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH)  </s>'

In [ ]:
# Define the local path to save the dataset
parquet_path = "/content/SQL-data.parquet"

In [ ]:
# Save the DataFrame to a Parquet file
table = pa.Table.from_pandas(df)
pq.write_table(table, parquet_path)

In [ ]:
# To access the hugging face
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [ ]:
# Uploading the dataset to Hugging Face
api = HfApi()
api.upload_file(
    path_or_fileobj=parquet_path,
    path_in_repo="Naturallanguate-sql.parquet",
    repo_id="prabal123/nl-sql-llama-2format",
    repo_type="dataset"
)
